In [9]:
# Importing libraries
import pandas as pd
from fuzzywuzzy import process, fuzz
def recommendation():
    
    # Read csv file into a pandas dataframe
    df = pd.read_csv("books.csv")
    df = df[df.language_code =='eng']
    df['Title'] = df['title'].str.split('(').str[0]
    print("Enter rating w/wo <rating> or random for book suggestion")
    search= input("enter to search ")
    if search=="random":
        df_new= df.sample(replace=True)[['Title', 'authors',"rating"]]
        print("Title: ", df_new[['Title']].to_string(index=False, header=False))
        print("Author: ", df_new[['authors']].to_string(index=False, header=False))
        print("Rating: ", df_new[['rating']].to_string(index=False, header=False))
        return "Enjoy!" 
    else:
        if "rating" in search:
            search=search.split()
            rate=float(search[1])
            
            if rate < 0 or rate > 5:
                print("Error: Enter rating between 0 and 5")
                return "Please Correct"
            else:
                df_rate = df[df['rating'].astype(float) >= rate]
                df_new= df_rate.sample(replace=True)[['Title', 'authors',"rating"]]
                print("Title: ", df_new[['Title']].to_string(index=False, header=False))
                print("Author: ", df_new[['authors']].to_string(index=False, header=False))
                print("Rating: ", df_new[['rating']].to_string(index=False, header=False))
                return "Enjoy!"
        if "author" in search:
            #Retrieve Author Name
            search = search.split()
            author = search[1:]
            author = " ".join(author)
            print(f"Entered Author: {author}")

            #Make Nested list with authors and subauthors to pass in fuzzywuzzy
            unique_authors = df['authors'].unique().tolist()
            for i in range(len(unique_authors)):
                subauthors = unique_authors[i].split('/')
                unique_authors[i] = subauthors
            #print(len(unique_authors))

            #Find Exact author names that fuzzily match user input 
            finds=[]
            for i in unique_authors:
                l = process.extract(author, i, scorer=fuzz.token_sort_ratio)
                for j in l:
                    if j[1] > 85: #change accuracy value for stricter or more general results
                        finds.append(i)          
            finds = ["/".join(l) for l in finds]            
            print(f"Authors found: {finds}")

            authors = tuple(df['authors'])
            for ix in range(len(authors)):
                if authors[ix] in finds:
                    df_new = df.iloc[[ix]]
                    print()
                    print("Title: ", df_new[['Title']].to_string(index=False, header=False))
                    print("Author: ", df_new[['authors']].to_string(index=False, header=False))
                    print("Rating: ", df_new[['rating']].to_string(index=False, header=False))

print(recommendation())

Enter rating w/wo <rating> or random for book suggestion
enter to search author jk rowling
Entered Author: jk rowling
Authors found: ['J.K. Rowling/Mary GrandPré', 'J.K. Rowling', 'Newt Scamander/J.K. Rowling']

Title:   Harry Potter and the Half-Blood Prince 
Author:   J.K. Rowling/Mary GrandPré
Rating:   4.57

Title:   Harry Potter and the Order of the Phoenix 
Author:   J.K. Rowling/Mary GrandPré
Rating:   4.49

Title:   Harry Potter and the Chamber of Secrets 
Author:   J.K. Rowling
Rating:   4.42

Title:   Harry Potter and the Prisoner of Azkaban 
Author:   J.K. Rowling/Mary GrandPré
Rating:   4.56

Title:   Harry Potter Boxed Set  Books 1-5 
Author:   J.K. Rowling/Mary GrandPré
Rating:   4.78

Title:   Harry Potter Collection 
Author:   J.K. Rowling
Rating:   4.73

Title:   Harry Potter Schoolbooks Box Set: Two Classic Books from the Library of Hogwarts School of Witchcraft and Wizardry
Author:   J.K. Rowling
Rating:   4.4

Title:   Harry Potter and the Half-Blood Prince 
Author: